In [1]:
import requests as rq
import csv
from bs4 import BeautifulSoup as bs
import os

In [2]:
class sport_scrape():
    """Parameter Breakdown (Parameter values should be inputted as spelled in the help section)
        -league: NHL or NBA (so far)
        -season: Regular or Playoff
        -form: NBA: Per: Per Game
                    Total: Season Totals
                    Advanced: Advanced Metrics
                    
                NHL: Skater: Basic Skater Season Stats
                     Advanced: Advanced Skater Season Stats
                     Goalie: Goalie Stats (no Advanced option for Goalies)
        -filepath: write out filepath to where you want files to be downloaded
        
        -parameter default values
            -league:NBA
            -season:Regular
            -form:Total
            -filepath:Current working directory
        
        There are no failsafes incase if you add a year out of bounds, so find a year range defined by reference.com
        
        DISCLAIMER: While scraping these sites should be okay (their policy is don't use bots that aggressively exceed the 
        activity that might be generated by a regular user). So if you're looking to scrape a few seasons of data, that's
        more than okay, as they allow csv "downloads" of their data. This script doesn't create any more calls to their
        servers than you would if you were copy and pasting the data from their site into an Excel Spreadsheet. If you 
        ignore this and use the script aggressively, your IP might be banned."""

    
    def __init__(self, start, end, season='Regular', league="NBA", filepath=os.getcwd(), form="Total"):
    
        self.start=start
        self.end=end
        self.league=league
        self.season=season
        self.years=range(self.start,self.end+1)
        self.filepath=filepath
        self.form=form
    
        self.url_switcher={'NBA':{"Regular":{"Total":("http://www.basketball-reference.com/leagues/NBA_%d_totals.html" % year for 
                                                      year in self.years),
                                             "Per":("http://www.basketball-reference.com/leagues/NBA_%d_per_game.html" % year for
                                                      year in self.years),
                                        "Advanced":("http://www.basketball-reference.com/leagues/NBA_%d_advanced.html" % year for 
                                                     year in self.years)},
                                 "Playoff":{"Total":("http://www.basketball-reference.com/playoffs/NBA_%d_totals.html" % year for 
                                               year in self.years),
                                            "Per":("http://www.basketball-reference.com/playoffs/NBA_%d_per_game.html" % year for
                                               year in self.years),
                                       "Advanced":("http://www.basketball-reference.com/playoffs/NBA_%d_advanced.html" % year for
                                               year in self.years)}},
                          "NHL":{"Regular":{"Skater":("http:/www.hockey-reference.com/leagues/NHL_%d_skaters.html" % year for
                                                      year in self.years),
                                           "Advanced":("http://www.hockey-reference.com/leagues/NHL_%d_skaters-advanced.html" %
                                                      year for year in self.years),
                                           "Goalie": ("http://www.hockey-reference.com/leagues/NHL_%d_goalies.html" % year for
                                                     year in self.years)},
                                "Playoff":{"Skater":("http:/www.hockey-reference.com/playoffs/NHL_%d_skaters.html" % year for
                                                      year in self.years),
                                           "Advanced":("http://www.hockey-reference.com/playoffs/NHL_%d_skaters-advanced.html" %
                                                      year for year in self.years),
                                           "Goalie": ("http://www.hockey-reference.com/playoffs/NHL_%d_goalies.html" % year for
                                                     year in self.years)}}}
        
        self.column_switcher={"NBA":{"Total": ['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', 
                                                 '3PA', '3P%', '2P', '2PA', '2P%','eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 
                                                 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
                                     "Per":['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
                                                '3PA', '3P%', '2P', '2PA', '2P%','eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
                                                'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
                                    "Advanced":['Player',"Pos",'Age',"Tm","G","MP","PER","TS%","3PAr","FTr","ORB%",
                                                "DRB%","TRB%","AST%","STL%","BLK%","TOV%","USG%","empty","OWS","DWS",
                                                "WS","WS/48","empty","OBPM","DBPM","BPM","VORP"]
        },
                             "NHL":{"Regular":{"Skater": ["Player","Pos","Age","Tm","GP","G","A","PTS","+/-","PIM","EV-G",
                                               "PP-G","SH-G","GW-G","EV-A","PP-A","SH-A","S","S%","TOI","ATOI",
                                               "BLK","HIT","FOW","FOL","FO%"],
                                     "Advanced":["Player","Pos","Age","Tm","GP","CF","CA","CF%","CF% rel","FF",
                                                 "FA","FF%","FF% rel","oiSH%","oiSV%","PDO","oZS%",'dZS%',"TOI/60",
                                                 "TOI(EV)","TK","GV", "E+/-", "SAtt.", "Thru%"],
                                    "Goalie":["Player","Age","Tm","GP","GS","W","L","T/O","GA","SA","SV","SV%","GAA",
                                              "SO","MIN","QS","QS%","RBS","GA%-","GSAA","G","A","PTS","PIM"]},
                                    "Playoff":{"Skater": ["Player","Pos","Age","Tm","GP","G","A","PTS","+/-","PIM","EV-G",
                                               "PP-G","SH-G","GW-G","EV-A","PP-A","SH-A","S","S%","TOI","ATOI",
                                               "BLK","HIT","FOW","FOL","FO%"],
                                     "Advanced":["Player","Pos","Age","Tm","GP","CF","CA","CF%","CF% rel","FF",
                                                 "FA","FF%","FF% rel","oiSH%","oiSV%","PDO","oZS%",'dZS%',"TOI/60",
                                                 "TOI(EV)","TK","GV", "E+/-", "SAtt.", "Thru%"],
                                    "Goalie":["Player","Age","Tm","GP","GS","W","L","GA","SA","SV","SV%","GAA",
                                              "SO","MIN","QS","QS%","RBS","GA%-","GSAA","G","A","PTS","PIM"]}
        }}
        
        self.name_switcher={"NBA":{"Regular":{"Total":"NBA_Season_Totals",
                                              "Per":"NBA_Season_PerGame",
                                              "Advanced":"NBA_Season_Advanced"},
                                  "Playoff":{"Total":"NBA_Playoff_Totals",
                                            "Per":"NBA_Playoff_PerGame",
                                            "Advanced":"NBA_Playoff_Advanced"}},
                           "NHL":{"Regular":{"Skater":"NHL_Skater_Season",
                                            "Advanced":"NHL_Skater_Season_Advanced",
                                            "Goalie":"NHL_Goalie_Season"},
                                 "Playoff":{"Skater":"NHL_Skater_Playoff",
                                           "Advanced":"NHL_Skater_Playoff_Advanced",
                                           "Goalie":"NHL_Goalie_Playoff"}}}
        
        self.urls=self.url_switcher[self.league][self.season][self.form]
        self.names=self.name_switcher[self.league][self.season][self.form]
        
        if self.league=="NHL":
            self.columns=self.column_switcher[self.league][self.season][self.form]
        else:
            self.columns=self.column_switcher[self.league][self.form]
    
    
    def write_csv(self,filepath, data):
        with open(filepath, 'w') as f:
            writer = csv.DictWriter(f, self.columns)
            writer.writeheader()
            writer.writerows(data)
    
    def nba_table_scraper2(self,soup,year):
        table=soup.findAll('table')[0]
        rows=table.find_all('tr')[1:]
        data = []
        for row in rows:
            cols = row.find_all('td')
            row_dict={}
            for col, name in zip(cols, self.columns):
                text = col.get_text()
                if text:
                    row_dict[name]=text
            if row_dict:
                data.append(row_dict)
        csv_filepath="%s/%s_%s.csv" % (self.filepath,self.names,year)
        self.write_csv(csv_filepath,data)
    
    def scraper(self):
        x=map(rq.get,self.urls)
        y=(page.text for page in x)
        z=(bs(text,"lxml") for text in y)
        for soup,year in zip(z,self.years):
            self.nba_table_scraper2(soup,year)

In [4]:
data = sport_scrape(start = 1977, end = 2019)
data.scraper()